In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import keras
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

DIM = (176,176)

work_dr = IDG()

train_ds = train_datagen.flow_from_directory(
        "/kaggle/input/knee-oa-dataset/Dataset/CLAHE/train",
        seed=123,
        target_size=(176, 176),
        batch_size=64,
        class_mode='sparse'
        )

test_ds = test_datagen.flow_from_directory(
"/kaggle/input/knee-oa-dataset/Dataset/CLAHE/test",
seed=123,
target_size=(176, 176),
batch_size=64,
    class_mode='sparse'
)

val_ds = val_datagen.flow_from_directory(
"/kaggle/input/knee-oa-dataset/Dataset/CLAHE/val",
seed=123,
target_size=(176, 176),
batch_size=64,
class_mode='sparse'
)

Found 5778 images belonging to 5 classes.
Found 1656 images belonging to 5 classes.
Found 826 images belonging to 5 classes.


In [4]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=10, 
                               restore_best_weights=True)

In [5]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

lr_schedule = LearningRateScheduler(lr_scheduler)

## ResNet152

In [6]:
from tensorflow.keras.applications import ResNet152
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

resnet152 = ResNet152(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
resnet152.trainable = True
resnet152_model_new = models.Sequential()
resnet152_model_new.add(resnet152)
resnet152_model_new.add(layers.GlobalAveragePooling2D())
resnet152_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
resnet152_model_new.add(layers.Dropout(0.3))
resnet152_model_new.add(layers.Dense(5, activation='softmax')) 

234698864/234698864 [==============================] - 1s 0us/step


In [7]:
resnet152_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [8]:
resnet152_new = resnet152_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 178s 775ms/step - loss: 2.8012 - accuracy: 0.4382 - val_loss: 2.0489 - val_accuracy: 0.3971 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 52s 563ms/step - loss: 1.5030 - accuracy: 0.4920 - val_loss: 1.7351 - val_accuracy: 0.2542 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 51s 557ms/step - loss: 1.2195 - accuracy: 0.5171 - val_loss: 2.1055 - val_accuracy: 0.1283 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 52s 561ms/step - loss: 1.1452 - accuracy: 0.5272 - val_loss: 2.5148 - val_accuracy: 0.1283 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 52s 566ms/step - loss: 1.1092 - accuracy: 0.5426 - val_loss: 2.4316 - val_accuracy: 0.1283 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 52s 566ms/step - loss: 1.0671 - accuracy: 0.5658 - val_loss: 1.6399 - val_accuracy: 0.2567 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 51s 556ms/s

In [9]:
loss, accuracy = resnet152_model_new.evaluate(test_ds)

26/26 [==============================] - 9s 356ms/step - loss: 0.7839 - accuracy: 0.6685


## ResNet101

In [ ]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

resnet101 = ResNet101(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
resnet101.trainable = True
resnet101_model_new = models.Sequential()
resnet101_model_new.add(resnet101)
resnet101_model_new.add(layers.GlobalAveragePooling2D())
resnet101_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
resnet101_model_new.add(layers.Dropout(0.3))
resnet101_model_new.add(layers.Dense(5, activation='softmax')) 

In [ ]:
resnet101_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [ ]:
resnet101_new = resnet101_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

In [ ]:
loss, accuracy = resnet101_model_new.evaluate(test_ds)

## ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
resnet50.trainable = True
resnet50_model_new = models.Sequential()
resnet50_model_new.add(resnet50)
resnet50_model_new.add(layers.GlobalAveragePooling2D())
resnet50_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
resnet50_model_new.add(layers.Dropout(0.3))
resnet50_model_new.add(layers.Dense(5, activation='softmax')) 

In [ ]:
resnet50_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [ ]:
resnet50_new = resnet50_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

In [ ]:
loss, accuracy = resnet50_model_new.evaluate(test_ds)

## DenseNet121

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

denseNet121 = DenseNet121(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
denseNet121.trainable = True
denseNet121_model_new = models.Sequential()
denseNet121_model_new.add(denseNet121)
denseNet121_model_new.add(layers.GlobalAveragePooling2D())
denseNet121_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
denseNet121_model_new.add(layers.Dropout(0.3))
denseNet121_model_new.add(layers.Dense(5, activation='softmax')) 

In [ ]:
denseNet121_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [ ]:
denseNet121_new = denseNet121_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

In [ ]:
loss, accuracy = denseNet121_model_new.evaluate(test_ds)

## MobileNetV2

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

mobileNetV2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
mobileNetV2.trainable = True
mobileNetV2_model_new = models.Sequential()
mobileNetV2_model_new.add(mobileNetV2)
mobileNetV2_model_new.add(layers.GlobalAveragePooling2D())
mobileNetV2_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
mobileNetV2_model_new.add(layers.Dropout(0.3))
mobileNetV2_model_new.add(layers.Dense(5, activation='softmax')) 

In [ ]:
mobileNetV2_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [ ]:
mobileNetV2_new = mobileNetV2_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

In [ ]:
loss, accuracy = mobileNetV2_model_new.evaluate(test_ds)

### InceptionV3

In [6]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.regularizers import l2

inceptionV3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(176, 176, 3))
inceptionV3.trainable = True
inceptionV3_model_new = models.Sequential()
inceptionV3_model_new.add(inceptionV3)
inceptionV3_model_new.add(layers.GlobalAveragePooling2D())
inceptionV3_model_new.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)))
inceptionV3_model_new.add(layers.Dropout(0.3))
inceptionV3_model_new.add(layers.Dense(5, activation='softmax')) 

87910968/87910968 [==============================] - 1s 0us/step


In [7]:
inceptionV3_model_new.compile(optimizer=optimizers.Adam(lr=0.0001), 
                           loss='sparse_categorical_crossentropy', 
                           metrics=['accuracy'])

In [8]:
inceptionV3_new = inceptionV3_model_new.fit(train_ds, epochs=50, validation_data=val_ds,
                       batch_size=64, callbacks = [early_stopping, lr_schedule])

Epoch 1/50
91/91 [==============================] - 117s 861ms/step - loss: 2.6448 - accuracy: 0.4251 - val_loss: 3.4355 - val_accuracy: 0.4044 - lr: 0.0010
Epoch 2/50
91/91 [==============================] - 48s 524ms/step - loss: 1.3867 - accuracy: 0.5220 - val_loss: 2.8365 - val_accuracy: 0.4528 - lr: 0.0010
Epoch 3/50
91/91 [==============================] - 47s 513ms/step - loss: 1.1334 - accuracy: 0.5481 - val_loss: 3.6425 - val_accuracy: 0.4516 - lr: 0.0010
Epoch 4/50
91/91 [==============================] - 47s 515ms/step - loss: 1.0446 - accuracy: 0.5760 - val_loss: 1.1053 - val_accuracy: 0.5860 - lr: 0.0010
Epoch 5/50
91/91 [==============================] - 47s 513ms/step - loss: 1.0034 - accuracy: 0.5924 - val_loss: 1.3170 - val_accuracy: 0.5545 - lr: 0.0010
Epoch 6/50
91/91 [==============================] - 47s 516ms/step - loss: 0.9605 - accuracy: 0.6061 - val_loss: 1.3897 - val_accuracy: 0.5484 - lr: 0.0010
Epoch 7/50
91/91 [==============================] - 47s 511ms/s

In [9]:
loss, accuracy = inceptionV3_model_new.evaluate(test_ds)

26/26 [==============================] - 14s 556ms/step - loss: 0.7702 - accuracy: 0.6757
